## Predicting the 2025 Super Bowl score using historical Super Bowl finalist data and resulting scores
The goal of this project is to predict the 2025 Super Bowl score using historical data. We selected the past fifteen Super Bowls as our dataset and pulled the data using the free SportsRadar API. We began by pulling all of the last fifteen Super Bowl winners and their statistics for the season that they competed in the Super Bowl. We then pulled the final scores of those Super Bowl games and trained multiple models to optimize our R squared and mean squared error (MSE). Models we tried include: Linear Regression, XGBoost, Random Forest Regression, and a small Neural Network. 
-Erik Van Cruyningen
-Bhuvan Balagar

## Imports

In [52]:
import pandas as pd
from time import sleep, time
import http.client
import requests
conn = http.client.HTTPSConnection("nfl-api-data.p.rapidapi.com")
SRAPIKey = 'aUqelqxEq2Yod9LThYDTKaQARtwgDp3UIvKRofSK'

## API Setup and Calls

pull all teams ids


In [ ]:
import requests

url = "https://api.sportradar.com/nfl/official/trial/v7/en/league/teams.json?api_key=aUqelqxEq2Yod9LThYDTKaQARtwgDp3UIvKRofSK"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)

{"league":{"id":"3c6d318a-6164-4290-9bbc-bf9bb21cc4b8","name":"National Football League","alias":"NFL"},"teams":[{"id":"f0e724b0-4cbf-495a-be47-013907608da9","name":"49ers","market":"San Francisco","alias":"SF","sr_id":"sr:competitor:4389"},{"id":"7b112545-38e6-483c-a55c-96cf6ee49cb8","name":"Bears","market":"Chicago","alias":"CHI","sr_id":"sr:competitor:4391"},{"id":"ad4ae08f-d808-42d5-a1e6-e9bc4e34d123","name":"Bengals","market":"Cincinnati","alias":"CIN","sr_id":"sr:competitor:4416"},{"id":"768c92aa-75ff-4a43-bcc0-f2798c2e1724","name":"Bills","market":"Buffalo","alias":"BUF","sr_id":"sr:competitor:4376"},{"id":"ce92bd47-93d5-4fe9-ada4-0fc681e6caa0","name":"Broncos","market":"Denver","alias":"DEN","sr_id":"sr:competitor:4418"},{"id":"d5a2eb42-8065-4174-ab79-0a6fa820e35e","name":"Browns","market":"Cleveland","alias":"CLE","sr_id":"sr:competitor:4417"},{"id":"4254d319-1bc7-4f81-b4ab-b5e6f3402b69","name":"Buccaneers","market":"Tampa Bay","alias":"TB","sr_id":"sr:competitor:4388"},{"id":

In [ ]:
# CONVERTS JSON RESPONSE TO LIST OF TEAMS AND IDs
dataTeam = response.json()  # Parse JSON response
dfTeam = pd.DataFrame(dataTeam["teams"])  # Convert 'teams' list to DataFrame
dfTeam.head()  # Show first few rows
simplified_df = dfTeam[['id', 'name']]
simplified_df

,id,name
0,f0e724b0-4cbf-495a-be47-013907608da9,49ers
1,7b112545-38e6-483c-a55c-96cf6ee49cb8,Bears
2,ad4ae08f-d808-42d5-a1e6-e9bc4e34d123,Bengals
3,768c92aa-75ff-4a43-bcc0-f2798c2e1724,Bills
4,ce92bd47-93d5-4fe9-ada4-0fc681e6caa0,Broncos
5,d5a2eb42-8065-4174-ab79-0a6fa820e35e,Browns
6,4254d319-1bc7-4f81-b4ab-b5e6f3402b69,Buccaneers
7,de760528-1dc0-416a-a978-b510d20692ff,Cardinals
8,1f6dcffb-9823-43cd-9ff4-e7a8466749b5,Chargers
9,6680d28d-d4d2-49f6-aace-5292d3ec02c2,Chiefs


In [ ]:
super_bowl_teams = {
    2010: ["0d855753-ea21-4953-89f9-0e20aff9eb73", "82cf9565-6eb9-4f01-bdbd-5aa0d472fcd9"],  # Saints vs. Colts
    2011: ["a20471b4-a8d9-40c7-95ad-90cc30e46932", "cb2f9f1f-ac67-424e-9e72-1475cb0ed398"],  # Packers vs. Steelers
    2012: ["04aa1c9d-66da-489d-b16a-1dee3f2eec4d", "97354895-8c77-4fd4-a860-32e62ea7382a"],  # Giants vs. Patriots
    2013: ["ebd87119-b331-4469-9ea6-d51fe3ce2f1c", "f0e724b0-4cbf-495a-be47-013907608da9"],  # Ravens vs. 49ers
    2014: ["3d08af9e-c767-4f88-a7dc-b920c6d2b4a8", "ce92bd47-93d5-4fe9-ada4-0fc681e6caa0"],  # Seahawks vs. Broncos
    2015: ["97354895-8c77-4fd4-a860-32e62ea7382a", "3d08af9e-c767-4f88-a7dc-b920c6d2b4a8"],  # Patriots vs. Seahawks
    2016: ["ce92bd47-93d5-4fe9-ada4-0fc681e6caa0", "f14bf5cc-9a82-4a38-bc15-d39f75ed5314"],  # Broncos vs. Panthers
    2017: ["97354895-8c77-4fd4-a860-32e62ea7382a", "e6aa13a4-0055-48a9-bc41-be28dc106929"],  # Patriots vs. Falcons
    2018: ["386bdbf9-9eea-4869-bb9a-274b0bc66e80", "97354895-8c77-4fd4-a860-32e62ea7382a"],  # Eagles vs. Patriots
    2019: ["97354895-8c77-4fd4-a860-32e62ea7382a", "2eff2a03-54d4-46ba-890e-2bc3925548f3"],  # Patriots vs. Rams
    2020: ["6680d28d-d4d2-49f6-aace-5292d3ec02c2", "f0e724b0-4cbf-495a-be47-013907608da9"],  # Chiefs vs. 49ers
    2021: ["4254d319-1bc7-4f81-b4ab-b5e6f3402b69", "6680d28d-d4d2-49f6-aace-5292d3ec02c2"],  # Buccaneers vs. Chiefs
    2022: ["2eff2a03-54d4-46ba-890e-2bc3925548f3", "ad4ae08f-d808-42d5-a1e6-e9bc4e34d123"],  # Rams vs. Bengals
    2023: ["6680d28d-d4d2-49f6-aace-5292d3ec02c2", "386bdbf9-9eea-4869-bb9a-274b0bc66e80"],  # Chiefs vs. Eagles
    2024: ["6680d28d-d4d2-49f6-aace-5292d3ec02c2", "f0e724b0-4cbf-495a-be47-013907608da9"],  # Chiefs vs. 49ers
}

# Example of accessing team IDs for a specific year
print(super_bowl_teams[2020])  # Output: ['6680d28d-d4d2-49f6-aace-5292d3ec02c2', 'f0e724b0-4cbf-495a-be47-013907608da9']


['6680d28d-d4d2-49f6-aace-5292d3ec02c2', 'f0e724b0-4cbf-495a-be47-013907608da9']


reducing team statistics api call to significant statistics

In [ ]:
import requests

url = "https://api.sportradar.com/nfl/official/trial/v7/en/seasons/2023/REG/teams/386bdbf9-9eea-4869-bb9a-274b0bc66e80/statistics.json?api_key=aUqelqxEq2Yod9LThYDTKaQARtwgDp3UIvKRofSK"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)

dataTeam = response.json()  # Parse JSON response
dfTeam = pd.json_normalize(dataTeam)
# dfTeam.columns()  # Show first few rows

{"id":"386bdbf9-9eea-4869-bb9a-274b0bc66e80","name":"Eagles","market":"Philadelphia","alias":"PHI","sr_id":"sr:competitor:4428","season":{"id":"a538d200-e916-4272-aa07-b486094a1668","year":2023,"type":"REG","name":"REG"},"record":{"games_played":17,"touchdowns":{"pass":24,"rush":22,"total_return":3,"total":49,"fumble_return":1,"int_return":2,"kick_return":0,"punt_return":0,"other":0},"rushing":{"avg_yards":4.294,"attempts":510,"touchdowns":22,"tlost":46,"tlost_yards":-109,"yards":2190,"longest":43,"longest_touchdown":12,"redzone_attempts":104,"broken_tackles":33,"kneel_downs":10,"scrambles":49,"yards_after_contact":744},"receiving":{"targets":525,"receptions":369,"avg_yards":11.022,"yards":4067,"touchdowns":24,"yards_after_catch":1786,"longest":63,"longest_touchdown":63,"redzone_targets":49,"air_yards":2281,"broken_tackles":25,"dropped_passes":19,"catchable_passes":18,"yards_after_contact":572},"punts":{"attempts":52,"yards":2537,"net_yards":2233,"blocked":0,"touchbacks":1,"inside_20":

getting seasonal statistics for each team

In [ ]:
dfTeamFinal = dfTeam
# dfTeamFinal = dfTeam.drop(columns = ['id', 'name', 'market', 'alias', 'sr_id', 'players', '_comment', 'season.id', 'season.year', 'season.type', 'season.name'])
dfTeamFinal.head()  # Show first few rows
dfTeamFinal_columns = list(dfTeamFinal.columns)

# superbowl_stats = pd.DataFrame(columns = dfTeamFinal_columns)
# superbowl_stats = pd.concat([superbowl_stats, dfTeamFinal], ignore_index=True)
# superbowl_stats.head()

dfTeamFinal.head()



,id,name,market,alias,sr_id,players,_comment,season.id,season.year,season.type,...,opponents.efficiency.goaltogo.pct,opponents.efficiency.redzone.attempts,opponents.efficiency.redzone.successes,opponents.efficiency.redzone.pct,opponents.efficiency.thirddown.attempts,opponents.efficiency.thirddown.successes,opponents.efficiency.thirddown.pct,opponents.efficiency.fourthdown.attempts,opponents.efficiency.fourthdown.successes,opponents.efficiency.fourthdown.pct
0,386bdbf9-9eea-4869-bb9a-274b0bc66e80,Eagles,Philadelphia,PHI,sr:competitor:4428,[{'id': '01f15239-cd2c-4bd7-be5d-5ba7eed41331'...,Generation started @ 2025-02-08 00:30:57 UTC e...,a538d200-e916-4272-aa07-b486094a1668,2023,REG,...,80.645,59,39,66.102,233,108,46.352,34,19,55.882


## Super Bowl Historical API Calls

In [ ]:
import requests
import time

url = f"https://api.sportradar.com/nfl/official/trial/v7/en/seasons/2024/REG/teams/0d855753-ea21-4953-89f9-0e20aff9eb73/statistics.json?api_key={SRAPIKey}"

headers = {"accept": "application/json"}

superbowl_stats = pd.DataFrame(columns = dfTeamFinal_columns)
for year, teams in super_bowl_teams.items():
    for team in teams:
        url = f"https://api.sportradar.com/nfl/official/trial/v7/en/seasons/{year}/REG/teams/{team}/statistics.json?api_key={SRAPIKey}"

        # Make the API request
        response = requests.get(url, headers=headers)

        # Check if request was successful
        if response.status_code == 200:
            dataTeam = response.json()  # Parse JSON response
            dfTeam = pd.json_normalize(dataTeam)
            # dfTeamFinal = dfTeam.drop(columns = ['id', 'name', 'market', 'alias', 'sr_id', 'players', '_comment', 'season.id', 'season.year', 'season.type', 'season.name'])
            superbowl_stats = pd.concat([superbowl_stats, dfTeam], ignore_index=True)
            print(f"Successfully retrieved stats for team {team} in {year}")
            # print(data)  # Print the retrieved data (or store it in a variable)
        else:
            print(f"Failed to fetch data for team {team} in {year}. Status code: {response.status_code}")
        time.sleep(5)

superbowl_stats.head()

Successfully retrieved stats for team 0d855753-ea21-4953-89f9-0e20aff9eb73 in 2010
Successfully retrieved stats for team 82cf9565-6eb9-4f01-bdbd-5aa0d472fcd9 in 2010
Successfully retrieved stats for team a20471b4-a8d9-40c7-95ad-90cc30e46932 in 2011
Successfully retrieved stats for team cb2f9f1f-ac67-424e-9e72-1475cb0ed398 in 2011
Successfully retrieved stats for team 04aa1c9d-66da-489d-b16a-1dee3f2eec4d in 2012
Successfully retrieved stats for team 97354895-8c77-4fd4-a860-32e62ea7382a in 2012
Successfully retrieved stats for team ebd87119-b331-4469-9ea6-d51fe3ce2f1c in 2013
Successfully retrieved stats for team f0e724b0-4cbf-495a-be47-013907608da9 in 2013
Successfully retrieved stats for team 3d08af9e-c767-4f88-a7dc-b920c6d2b4a8 in 2014
Successfully retrieved stats for team ce92bd47-93d5-4fe9-ada4-0fc681e6caa0 in 2014
Successfully retrieved stats for team 97354895-8c77-4fd4-a860-32e62ea7382a in 2015
Successfully retrieved stats for team 3d08af9e-c767-4f88-a7dc-b920c6d2b4a8 in 2015
Succ

,id,name,market,alias,sr_id,players,_comment,season.id,season.year,season.type,...,record.misc_returns.ez_rec_touchdowns,record.misc_returns.returns,opponents.misc_returns.yards,opponents.misc_returns.touchdowns,opponents.misc_returns.longest_touchdown,opponents.misc_returns.blk_fg_touchdowns,opponents.misc_returns.blk_punt_touchdowns,opponents.misc_returns.fg_return_touchdowns,opponents.misc_returns.ez_rec_touchdowns,opponents.misc_returns.returns
0,0d855753-ea21-4953-89f9-0e20aff9eb73,Saints,New Orleans,NO,sr:competitor:4425,[{'id': '002a7fbe-9272-4c85-b7a8-d4b13c7f1f65'...,Generation started @ 2025-02-08 00:31:01 UTC e...,854cc6a0-3fdc-4239-b950-191672a5dbee,2010,REG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,82cf9565-6eb9-4f01-bdbd-5aa0d472fcd9,Colts,Indianapolis,IND,sr:competitor:4421,[{'id': '07e7d555-9588-4c29-a6cb-481dea46bd52'...,Generation started @ 2025-02-08 00:31:06 UTC e...,854cc6a0-3fdc-4239-b950-191672a5dbee,2010,REG,...,0.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,a20471b4-a8d9-40c7-95ad-90cc30e46932,Packers,Green Bay,GB,sr:competitor:4420,[{'id': '034b3ecd-3827-40f3-ab31-198106fde215'...,Generation started @ 2025-02-08 00:31:12 UTC e...,01335d6d-e8f5-4b55-b803-7ee187ad2d00,2011,REG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,cb2f9f1f-ac67-424e-9e72-1475cb0ed398,Steelers,Pittsburgh,PIT,sr:competitor:4345,[{'id': '013576c7-11b4-403d-b80f-c918d9b7a1cd'...,Generation started @ 2025-02-08 00:31:17 UTC e...,01335d6d-e8f5-4b55-b803-7ee187ad2d00,2011,REG,...,0.0,1.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
4,04aa1c9d-66da-489d-b16a-1dee3f2eec4d,Giants,New York,NYG,sr:competitor:4426,[{'id': '05492ae2-c310-49b2-927c-58d7d2d7d20c'...,Generation started @ 2025-02-08 00:31:23 UTC e...,1adca71b-202f-44d5-9fab-abd45d813954,2012,REG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
superbowl_stats.to_csv('superbowl_stats.csv', index=False)
# superbowl_stats_backup =
dfTeam.head()

,id,name,market,alias,sr_id,players,_comment,season.id,season.year,season.type,...,opponents.efficiency.goaltogo.pct,opponents.efficiency.redzone.attempts,opponents.efficiency.redzone.successes,opponents.efficiency.redzone.pct,opponents.efficiency.thirddown.attempts,opponents.efficiency.thirddown.successes,opponents.efficiency.thirddown.pct,opponents.efficiency.fourthdown.attempts,opponents.efficiency.fourthdown.successes,opponents.efficiency.fourthdown.pct
0,f0e724b0-4cbf-495a-be47-013907608da9,49ers,San Francisco,SF,sr:competitor:4389,[{'id': '014613d0-057f-11ef-b81d-51c506ab6dea'...,Generation started @ 2025-02-08 00:33:43 UTC e...,26a67dd0-dfc9-11ee-8f7c-27de425212d4,2024,REG,...,85.714,57,39,68.421,216,93,43.056,26,17,65.385


In [ ]:
superbowl_stats.shape

superbowlUntouched = superbowl_stats
#do not use untouched.

data = {

    "Score": [
        31, 17,
        31, 25,
        21, 17,
        34, 31,
        43, 8,
        28, 24,
        24, 10,
        34, 28,
        41, 33,
        13, 3,
        31, 20,
        31, 9,
        23, 20,
        38, 35,
        25, 22
    ]
}

df_scores = pd.DataFrame(data)
df_scores.head()

superbowl_stats['Score'] = df_scores
superbowl_stats[['name', 'Score']].head(31)



,name,Score
0,Saints,31
1,Colts,17
2,Packers,31
3,Steelers,25
4,Giants,21
5,Patriots,17
6,Ravens,34
7,49ers,31
8,Seahawks,43
9,Broncos,8


In [ ]:
superbowl_stats.head()
#superbowl_stats.to_csv('superbowl_stats.csv', index=False)

,id,name,market,alias,sr_id,players,_comment,season.id,season.year,season.type,...,record.misc_returns.returns,opponents.misc_returns.yards,opponents.misc_returns.touchdowns,opponents.misc_returns.longest_touchdown,opponents.misc_returns.blk_fg_touchdowns,opponents.misc_returns.blk_punt_touchdowns,opponents.misc_returns.fg_return_touchdowns,opponents.misc_returns.ez_rec_touchdowns,opponents.misc_returns.returns,Score
0,0d855753-ea21-4953-89f9-0e20aff9eb73,Saints,New Orleans,NO,sr:competitor:4425,[{'id': '002a7fbe-9272-4c85-b7a8-d4b13c7f1f65'...,Generation started @ 2025-02-08 00:31:01 UTC e...,854cc6a0-3fdc-4239-b950-191672a5dbee,2010,REG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31
1,82cf9565-6eb9-4f01-bdbd-5aa0d472fcd9,Colts,Indianapolis,IND,sr:competitor:4421,[{'id': '07e7d555-9588-4c29-a6cb-481dea46bd52'...,Generation started @ 2025-02-08 00:31:06 UTC e...,854cc6a0-3fdc-4239-b950-191672a5dbee,2010,REG,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17
2,a20471b4-a8d9-40c7-95ad-90cc30e46932,Packers,Green Bay,GB,sr:competitor:4420,[{'id': '034b3ecd-3827-40f3-ab31-198106fde215'...,Generation started @ 2025-02-08 00:31:12 UTC e...,01335d6d-e8f5-4b55-b803-7ee187ad2d00,2011,REG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31
3,cb2f9f1f-ac67-424e-9e72-1475cb0ed398,Steelers,Pittsburgh,PIT,sr:competitor:4345,[{'id': '013576c7-11b4-403d-b80f-c918d9b7a1cd'...,Generation started @ 2025-02-08 00:31:17 UTC e...,01335d6d-e8f5-4b55-b803-7ee187ad2d00,2011,REG,...,1.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,25
4,04aa1c9d-66da-489d-b16a-1dee3f2eec4d,Giants,New York,NYG,sr:competitor:4426,[{'id': '05492ae2-c310-49b2-927c-58d7d2d7d20c'...,Generation started @ 2025-02-08 00:31:23 UTC e...,1adca71b-202f-44d5-9fab-abd45d813954,2012,REG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21


## Data Cleaning and Pre-Processing

In [53]:
# from google.colab import drive
# drive.mount('/content/drive')

#"/content/drive/MyDrive/SportsBettingModel/Data/superbowl_stats.csv"
#superbowl_stats = pd.read_csv('/content/drive/MyDrive/SportsBettingModel/Data/superbowl_stats.csv')
superbowl_stats = pd.read_csv('C:/Users/erikv/Desktop/JuniorYear/MardellBettingInc/data/superbowl_stats.csv')

In [54]:
superbowl_stats.head(2)

,id,name,market,alias,sr_id,players,_comment,season.id,season.year,season.type,...,record.misc_returns.returns,opponents.misc_returns.yards,opponents.misc_returns.touchdowns,opponents.misc_returns.longest_touchdown,opponents.misc_returns.blk_fg_touchdowns,opponents.misc_returns.blk_punt_touchdowns,opponents.misc_returns.fg_return_touchdowns,opponents.misc_returns.ez_rec_touchdowns,opponents.misc_returns.returns,Score
0,0d855753-ea21-4953-89f9-0e20aff9eb73,Saints,New Orleans,NO,sr:competitor:4425,[{'id': '002a7fbe-9272-4c85-b7a8-d4b13c7f1f65'...,Generation started @ 2025-02-08 00:31:01 UTC e...,854cc6a0-3fdc-4239-b950-191672a5dbee,2010,REG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31
1,82cf9565-6eb9-4f01-bdbd-5aa0d472fcd9,Colts,Indianapolis,IND,sr:competitor:4421,[{'id': '07e7d555-9588-4c29-a6cb-481dea46bd52'...,Generation started @ 2025-02-08 00:31:06 UTC e...,854cc6a0-3fdc-4239-b950-191672a5dbee,2010,REG,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17


In [87]:
#More imports
#imports
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import tensorflow as tf
from scipy.stats import norm


# modelling
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelBinarizer #Z-score variables
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import cross_val_score



# performance
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error, mean_absolute_error, accuracy_score #model evaluation
from sklearn.model_selection import KFold, GridSearchCV


# model validation imports
from sklearn.model_selection import train_test_split # simple TT split cv

# pipeline imports
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import make_column_transformer

In [ ]:
#ATP, Can do a prediction model to predict the most likely end score for the Chiefs/Eagles game? One model for Chiefs score, one for Eagles score?




# EAGLES DATA SETUP
# X values are all the predictors, Y value is the end score
superbowl_stats = superbowl_stats.fillna(0)
superbowl_stats.reset_index(inplace = True)


superbowl_stats_predictors = superbowl_stats.drop(columns = ['id', 'name', 'market', 'alias', 'sr_id', 'players', '_comment', 'season.id', 'season.year', 'season.type', 'season.name', 'Score'])




X = superbowl_stats_predictors
y = superbowl_stats['Score']


X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2)


#Theory on Pre:
#   Scaling the values in a z-score does not make sense because a large amount of values are rare events.
#   For example: Touchdowns scored from a muffed field goal attempt may be 0 for all but one team.
#   When this value is z-scored, it puts too much importance on one variable and results in a poor overall model
# #   Not z-scoring the values ends with a more accurate model so far.
# pre = make_column_transformer((StandardScaler(), list(superbowl_stats_predictors.columns)),
#                            remainder = "passthrough").set_output(transform="pandas")




## XGBoost Regression Model. Best Results: 0.35 R2, 58.5 MSE

In [53]:
#XGBoost Regression Model Best Stats:
# Train MSE :  44.8903694152832
# Train MAE :  5.1429219245910645
# Train MAPE:  0.4564107358455658
# Train R2  :  0.5127140283584595
# Test MSE  :  128.366455078125
# Test MAE  :  9.960031509399414
# Test MAPE :  0.4543425738811493
# Test R2  :  -0.42938220500946045

#linear regression
#lr = LinearRegression()
# model = XGBRegressor(
#     objective='reg:squarederror',
#     random_state=42,
#     colsample_bytree=0.8,
#     gamma=0,
#     learning_rate=0.01,
#     max_depth=3,
#     n_estimators=50,
#     subsample=1.0
# )
model = XGBRegressor(objective='reg:squarederror', random_state=42)

# pipe = Pipeline([("pre", pre),
#                 ("xgbregressor", model)])

#Best Parameters: {'xgbregressor__colsample_bytree': 0.8, 'xgbregressor__gamma': 0, 'xgbregressor__learning_rate': 0.01, 'xgbregressor__max_depth': 3, 
# 'xgbregressor__n_estimators': 50, 'xgbregressor__subsample': 1.0}

#GRID SEARCH FOR HYPERPARAMS
param_grid = {
    "xgbregressor__n_estimators": [50, 100, 200],  # Number of boosting rounds
    "xgbregressor__learning_rate": [0.01, 0.05, 0.1],  # Step size shrinkage
    "xgbregressor__max_depth": [3, 5, 7],  # Depth of each tree
    "xgbregressor__subsample": [0.6, 0.8, 1.0],  # Fraction of samples used per tree
    "xgbregressor__colsample_bytree": [0.6, 0.8, 1.0],  # Fraction of features per tree
    "xgbregressor__gamma": [0, 0.1, 0.2]  # Minimum loss reduction
}

grid_search = GridSearchCV(model, param_grid, scoring='neg_mean_squared_error', cv=5, verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Print best parameters
print("Best Parameters:", grid_search.best_params_)

# Use the best model
best_model = grid_search.best_estimator_

y_pred_train = best_model.predict(X_train)
y_pred_test = best_model.predict(X_test)


# pipe.fit(X_train,y_train)

# y_pred_train = pipe.predict(X_train)
# y_pred_test = pipe.predict(X_test)


print("Train MSE : ", mean_squared_error(y_train,y_pred_train))
print("Train MAE : ", mean_absolute_error(y_train,y_pred_train))
print("Train MAPE: ", mean_absolute_percentage_error(y_train,y_pred_train))
print("Train R2  : ", r2_score(y_train,y_pred_train))

print("Test MSE  : ", mean_squared_error(y_test,y_pred_test))
print("Test MAE  : ", mean_absolute_error(y_test,y_pred_test))
print("Test MAPE : ", mean_absolute_percentage_error(y_test,y_pred_test))
print("Test R2  : ", r2_score(y_test,y_pred_test))

Fitting 5 folds for each of 729 candidates, totalling 3645 fits
Best Parameters: {'xgbregressor__colsample_bytree': 0.6, 'xgbregressor__gamma': 0, 'xgbregressor__learning_rate': 0.01, 'xgbregressor__max_depth': 3, 'xgbregressor__n_estimators': 50, 'xgbregressor__subsample': 0.6}
Train MSE :  1.62562344030448e-07
Train MAE :  0.00026229023933410645
Train MAPE:  2.605903137009591e-05
Train R2  :  1.0
Test MSE  :  58.4691162109375
Test MAE  :  6.664147853851318
Test MAPE :  0.47521886229515076
Test R2  :  0.35194331407546997


## XGBoost Feature Reduction Best Results: 0.485 R2, 34.72 MSE With 27 Features

In [ ]:
#New XGBoost model to find the top 100 features
xgb = XGBRegressor(objective="reg:squarederror", random_state=42)
xgb.fit(X_train, y_train)

#Create DF with all the new features and their importances
feature_importances = pd.DataFrame({
    'feature': X_train.columns,
    'importance': xgb.feature_importances_
})

#Sort features by importance
feature_importances = feature_importances.sort_values(by="importance", ascending=False)

#Grab only the top 100 features
top_features = feature_importances.head(27)["feature"].tolist() #27 FEATURES IS BEST

X_train_selected = X_train[top_features]
X_test_selected = X_test[top_features]

print(top_features)

# 100 Features: 0.42 R2, 51.5 MSE
# 50 Features:  0.42 R2, 51.5 MSE
# 30 Features   0.44 R2, 50.7 MSE
# 29 Features:  0.421 R2, 52.22 MSE
# 28 Features:  0.423 R2, 52.02 MSE

# 27 Features:  0.473 R2, 47.54 MSE #BEST MODEL

# 26 Features:  0.446 R2, 49.97 MSE
# 25 Features:  0.446 R2, 49.97 MSE
# 20 Features:  0.34 R2, 59.1 MSE
# 10 Features:  -0.22 R2, 110.3 MSE

['opponents.extra_points.kicks.attempts', 'record.receiving.targets', 'record.punt_returns.longest', 'record.kickoffs.touchbacks', 'record.defense.sp_assists', 'record.efficiency.goaltogo.successes', 'opponents.field_goals.avg_yards', 'record.defense.passes_defended', 'opponents.passing.throw_aways', 'opponents.fumbles.opp_rec_yards', 'opponents.extra_points.kicks.made', 'record.rushing.attempts', 'opponents.rushing.yards_after_contact', 'record.fumbles.own_rec', 'record.touchdowns.total', 'opponents.touchdowns.fumble_return', 'record.touchdowns.fumble_return', 'record.penalties.penalties', 'record.efficiency.goaltogo.attempts', 'record.touchdowns.rush', 'record.efficiency.redzone.successes', 'record.rushing.tlost', 'record.misc_returns.longest_touchdown', 'record.rushing.longest', 'opponents.punts.avg_net_yards', 'opponents.rushing.yards', 'record.rushing.broken_tackles']


In [ ]:
#New XGBoost model using only the top N features - With hyperparam tuning

#kf = KFold(n_splits = 24, shuffle = True, random_state = 37)


xgbTest = XGBRegressor( random_state = 42)


#GRID SEARCH FOR HYPERPARAMS
param_grid = {
    "xgbregressor__n_estimators": [50, 100, 200],  # Number of boosting rounds
    "xgbregressor__learning_rate": [0.01, 0.05, 0.1],  # Step size shrinkage
    "xgbregressor__max_depth": [3, 5, 7],  # Depth of each tree
    "xgbregressor__subsample": [0.6, 0.8, 1.0],  # Fraction of samples used per tree
    "xgbregressor__colsample_bytree": [0.6, 0.8, 1.0],  # Fraction of features per tree
    "xgbregressor__gamma": [0, 0.1, 0.2]  # Minimum loss reduction
}

grid_search = GridSearchCV(xgbTest, param_grid, scoring='neg_mean_squared_error', cv=5, verbose=1, n_jobs=-1)
grid_search.fit(X_train_selected, y_train)

# Print best parameters
print("Best Parameters:", grid_search.best_params_)

# # Use the best model
best_model = grid_search.best_estimator_

y_pred_train = best_model.predict(X_train_selected)
y_pred_test = best_model.predict(X_test_selected)

print("Train MSE : ", mean_squared_error(y_train,y_pred_train))
print("Train MAE : ", mean_absolute_error(y_train,y_pred_train))
print("Train MAPE: ", mean_absolute_percentage_error(y_train,y_pred_train))
print("Train R2  : ", r2_score(y_train,y_pred_train))

print("Test MSE  : ", mean_squared_error(y_test,y_pred_test))
print("Test MAE  : ", mean_absolute_error(y_test,y_pred_test))
print("Test MAPE : ", mean_absolute_percentage_error(y_test,y_pred_test))
print("Test R2  : ", r2_score(y_test,y_pred_test))

#Something weird happened: 0.698 R2, with a 20.05 MSE


Fitting 24 folds for each of 729 candidates, totalling 17496 fits
Best Parameters: {'xgbregressor__colsample_bytree': 0.6, 'xgbregressor__gamma': 0, 'xgbregressor__learning_rate': 0.01, 'xgbregressor__max_depth': 3, 'xgbregressor__n_estimators': 50, 'xgbregressor__subsample': 0.6}
Train MSE :  2.0378092813189141e-07
Train MAE :  0.00031541785574518144
Train MAPE:  2.9482427635230124e-05
Train R2  :  1.0
Test MSE  :  34.72813415527344
Test MAE  :  5.432941436767578
Test MAPE :  0.3032127320766449
Test R2  :  0.4859321713447571


In [ ]:
#Save the model
import pickle
# save
with open('model.pkl','wb') as f:
    pickle.dump(best_model,f)
    

## Random Forest Regression. Best Results: -0.20 R2, 108.7 MSE

In [75]:
#Random Forest Regression
tree = RandomForestRegressor()

# pipe = Pipeline([("pre", pre),
#                 ("RFRegressor", tree)])

param_grid = {
    "n_estimators": [50, 100, 200],  # Number of trees
    "max_depth": [None, 10, 20, 30],  # Max depth of each tree
    "min_samples_split": [2, 5, 10],  # Min samples required to split a node
    "min_samples_leaf": [1, 2, 4],  # Min samples per leaf
    "max_features": ['auto', 'sqrt'],  # Number of features per split
    "bootstrap": [True, False]  # Whether to use bootstrapping
}
#['bootstrap', 'ccp_alpha', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'max_samples', 'min_impurity_decrease', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'monotonic_cst', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start']

grid_search = GridSearchCV(tree, param_grid, scoring='neg_mean_squared_error', cv=5, verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

print("Best Parameters:", grid_search.best_params_)

best_model = grid_search.best_estimator_

# pipe.fit(X_train,y_train)

y_pred_train = best_model.predict(X_train)
y_pred_test = best_model.predict(X_test)


print("Train MSE : ", mean_squared_error(y_train,y_pred_train))
print("Train MAE : ", mean_absolute_error(y_train,y_pred_train))
print("Train MAPE: ", mean_absolute_percentage_error(y_train,y_pred_train))
print("Train R2  : ", r2_score(y_train,y_pred_train))

print("Test MSE  : ", mean_squared_error(y_test,y_pred_test))
print("Test MAE  : ", mean_absolute_error(y_test,y_pred_test))
print("Test MAPE : ", mean_absolute_percentage_error(y_test,y_pred_test))
print("Test R2  : ", r2_score(y_test,y_pred_test))

Fitting 5 folds for each of 432 candidates, totalling 2160 fits
Best Parameters: {'bootstrap': True, 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 5, 'n_estimators': 50}
Train MSE :  43.990730983735524
Train MAE :  5.101980202667701
Train MAPE:  0.44677160994082893
Train R2  :  0.5380203280587867
Test MSE  :  108.70833505297912
Test MAE  :  9.135302632552635
Test MAPE :  0.673273305442451
Test R2  :  -0.20489533925715797


## Random Forest Regression Feature Reduction. Best Results: -0.003 R2, 90.49 MSE With 50 Features

In [ ]:
#Feature Selection Using Random Forest Regression
treeSelect = RandomForestRegressor()
treeSelect.fit(X_train, y_train)

#Create DF with all the new features and their importances
feature_importances = pd.DataFrame({
    'feature': X_train.columns,
    'importance': treeSelect.feature_importances_
})

#Sort features by importance
feature_importances = feature_importances.sort_values(by="importance", ascending=False)

#Grab only the top 100 features
top_features = feature_importances.head(75)["feature"].tolist() #27 FEATURES IS BEST

X_train_selected = X_train[top_features]
X_test_selected = X_test[top_features]

print(top_features)

#Random Forest Regression With Feature Importance
tree = RandomForestRegressor()

#List of hyperparameters to change in the random forest
param_grid = {
    "n_estimators": [50, 100, 200],  # Number of trees
    "max_depth": [None, 10, 20, 30],  # Max depth of each tree
    "min_samples_split": [2, 5, 10],  # Min samples required to split a node
    "min_samples_leaf": [1, 2, 4],  # Min samples per leaf
    "max_features": ['auto', 'sqrt'],  # Number of features per split
    "bootstrap": [True, False]  # Whether to use bootstrapping
}

#Find the hyperparams
grid_search = GridSearchCV(tree, param_grid, scoring='neg_mean_squared_error', cv=5, verbose=1, n_jobs=-1)
grid_search.fit(X_train_selected, y_train)

print("Best Parameters:", grid_search.best_params_)

#use our new hyperparam model
best_model = grid_search.best_estimator_

#Test it
y_pred_train = best_model.predict(X_train_selected)
y_pred_test = best_model.predict(X_test_selected)


print("Train MSE : ", mean_squared_error(y_train,y_pred_train))
print("Train MAE : ", mean_absolute_error(y_train,y_pred_train))
print("Train MAPE: ", mean_absolute_percentage_error(y_train,y_pred_train))
print("Train R2  : ", r2_score(y_train,y_pred_train))

print("Test MSE  : ", mean_squared_error(y_test,y_pred_test))
print("Test MAE  : ", mean_absolute_error(y_test,y_pred_test))
print("Test MAPE : ", mean_absolute_percentage_error(y_test,y_pred_test))
print("Test R2  : ", r2_score(y_test,y_pred_test))

#100 Features: -0.359 R2, 122.65 MSE
#75 Features:  -0.182 R2, 106.67 MSE
#50 Features:  -0.003 R2, 90.49 MSE
#30 Features:  -0.386 R2, 125.09 MSE

['record.punt_returns.longest', 'opponents.punts.avg_net_yards', 'opponents.extra_points.kicks.attempts', 'record.field_goals.longest', 'record.efficiency.goaltogo.pct', 'opponents.punts.avg_yards', 'opponents.passing.pocket_time', 'opponents.rushing.yards_after_contact', 'opponents.defense.sp_blocks', 'opponents.fumbles.opp_rec_yards', 'record.passing.cmp_pct', 'record.defense.combined', 'record.defense.tloss', 'opponents.rushing.yards', 'record.defense.knockdowns', 'record.touchdowns.fumble_return', 'record.fumbles.opp_rec_yards', 'record.defense.misc_fumble_recoveries', 'opponents.touchdowns.total', 'opponents.rushing.attempts', 'opponents.passing.hurries', 'record.defense.fumble_recoveries', 'opponents.passing.poor_throws', 'record.receiving.targets', 'record.efficiency.goaltogo.successes', 'record.efficiency.fourthdown.attempts', 'record.passing.pocket_time', 'record.kickoffs.touchbacks', 'opponents.kickoffs.inside_20', 'record.defense.sack_yards', 'record.rushing.longest_touchdow

## Neural Network using TensorFlow - -1.91 R2, 302.42 MSE - Current bug is 'super' object has no attribute '__sklearn_tags__'

In [15]:
def create_model(learning_rate=0.01, neurons=64, layers=2, dropout_rate=0.0):
    model = Sequential()
    model.add(Dense(neurons, activation='relu', input_shape=(X_train.shape[1],)))  # First layer

    # Add more layers dynamically based on "layers" parameter
    for _ in range(layers - 1):
        model.add(Dense(neurons, activation='relu'))
        if dropout_rate > 0:
            model.add(Dropout(dropout_rate))  # Dropout for regularization

    model.add(Dense(1))  # Output layer (Regression)
    
    # Compile model
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), 
                  loss='mean_squared_error', 
                  metrics=['mean_squared_error'])
    
    return model

In [16]:
model = KerasRegressor(build_fn=create_model, verbose=0)

param_grid = {
    'learning_rate': [0.001, 0.01, 0.1],  # Different learning rates
    'neurons': [32, 64, 128],  # Number of neurons per layer
    'layers': [2, 3, 4],  # Number of hidden layers
    'dropout_rate': [0.0, 0.2, 0.5],  # Dropout to prevent overfitting
    'batch_size': [16, 32, 64],  # Number of samples per batch
    'epochs': [50, 100, 200]  # Number of times the model sees the dataset
}

grid = GridSearchCV(estimator=model, param_grid=param_grid, 
                    scoring='neg_mean_squared_error', cv=3, verbose=1, n_jobs=-1)

grid_result = grid.fit(X_train, y_train)

print("Best Parameters:", grid_result.best_params_)
print("Best Score:", grid_result.best_score_)

# model = keras.Sequential([
#     keras.layers.Dense(64, activation='relu'),
#     keras.layers.Dense(32, activation='relu'),
#     keras.layers.Dense(1)  # Predicting a single score value
# ])

model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, epochs=100, batch_size=8)
predictions = model.predict(X_test)

y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

print("Train MSE : ", mean_squared_error(y_train,y_pred_train))
print("Train MAE : ", mean_absolute_error(y_train,y_pred_train))
print("Train MAPE: ", mean_absolute_percentage_error(y_train,y_pred_train))
print("Train R2  : ", r2_score(y_train,y_pred_train))

print("Test MSE  : ", mean_squared_error(y_test,y_pred_test))
print("Test MAE  : ", mean_absolute_error(y_test,y_pred_test))
print("Test MAPE : ", mean_absolute_percentage_error(y_test,y_pred_test))
print("Test R2  : ", r2_score(y_test,y_pred_test))



AttributeError: 'super' object has no attribute '__sklearn_tags__'

## API Calls for the Chiefs and Eagles Super Bowl

In [80]:
#pull the current season stats for the Super Bowl teams
# 1. Chiefs

#Chiefs: 6680d28d-d4d2-49f6-aace-5292d3ec02c2
#Eagles: 386bdbf9-9eea-4869-bb9a-274b0bc66e80

year = 2024
team = '6680d28d-d4d2-49f6-aace-5292d3ec02c2'


url = f"https://api.sportradar.com/nfl/official/trial/v7/en/seasons/{year}/REG/teams/{team}/statistics.json?api_key={SRAPIKey}"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

chiefs = response.json()  # Parse JSON response
df2024Chiefs = pd.json_normalize(chiefs)
df2024Chiefs.head()


# dfTeam.columns()  # Show first few rows

,id,name,market,alias,sr_id,players,_comment,season.id,season.year,season.type,...,opponents.efficiency.goaltogo.pct,opponents.efficiency.redzone.attempts,opponents.efficiency.redzone.successes,opponents.efficiency.redzone.pct,opponents.efficiency.thirddown.attempts,opponents.efficiency.thirddown.successes,opponents.efficiency.thirddown.pct,opponents.efficiency.fourthdown.attempts,opponents.efficiency.fourthdown.successes,opponents.efficiency.fourthdown.pct
0,6680d28d-d4d2-49f6-aace-5292d3ec02c2,Chiefs,Kansas City,KC,sr:competitor:4422,[{'id': '0ef0d0ca-2d2d-455b-ab63-a20c01303e37'...,Generation started @ 2025-02-08 20:50:28 UTC e...,26a67dd0-dfc9-11ee-8f7c-27de425212d4,2024,REG,...,68.966,54,28,51.852,215,93,43.256,23,12,52.174


In [81]:
#2 Eagles
team = '386bdbf9-9eea-4869-bb9a-274b0bc66e80'
url = f"https://api.sportradar.com/nfl/official/trial/v7/en/seasons/{year}/REG/teams/{team}/statistics.json?api_key={SRAPIKey}"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

eagles = response.json()  # Parse JSON response
df2024= pd.json_normalize(eagles)
df2024.head()

,id,name,market,alias,sr_id,players,_comment,season.id,season.year,season.type,...,opponents.efficiency.goaltogo.pct,opponents.efficiency.redzone.attempts,opponents.efficiency.redzone.successes,opponents.efficiency.redzone.pct,opponents.efficiency.thirddown.attempts,opponents.efficiency.thirddown.successes,opponents.efficiency.thirddown.pct,opponents.efficiency.fourthdown.attempts,opponents.efficiency.fourthdown.successes,opponents.efficiency.fourthdown.pct
0,386bdbf9-9eea-4869-bb9a-274b0bc66e80,Eagles,Philadelphia,PHI,sr:competitor:4428,[{'id': '01f15239-cd2c-4bd7-be5d-5ba7eed41331'...,Generation started @ 2025-02-08 20:50:35 UTC e...,26a67dd0-dfc9-11ee-8f7c-27de425212d4,2024,REG,...,76.0,48,24,50.0,211,75,35.545,30,19,63.333


In [82]:
#Merge the two finalist's dataframes
df2024 = pd.concat([df2024, df2024Chiefs], ignore_index=True)
df2024.head()

# Normalize data as the train model was
SBFinalists = df2024.drop(columns = ['id', 'name', 'market', 'alias', 'sr_id', 'players', '_comment', 'season.id', 'season.year', 'season.type', 'season.name'])
SBFinalists.head()



,record.games_played,record.touchdowns.pass,record.touchdowns.rush,record.touchdowns.total_return,record.touchdowns.total,record.touchdowns.fumble_return,record.touchdowns.int_return,record.touchdowns.kick_return,record.touchdowns.punt_return,record.touchdowns.other,...,opponents.efficiency.fourthdown.successes,opponents.efficiency.fourthdown.pct,record.misc_returns.yards,record.misc_returns.touchdowns,record.misc_returns.longest_touchdown,record.misc_returns.blk_fg_touchdowns,record.misc_returns.blk_punt_touchdowns,record.misc_returns.fg_return_touchdowns,record.misc_returns.ez_rec_touchdowns,record.misc_returns.returns
0,17,24,29,1,54,0,1,0,0,0,...,19,63.333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,17,26,15,1,42,1,0,0,0,0,...,12,52.174,57.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0


In [ ]:
#Predict using final model
# load the model
with open('0.485R2.pkl', 'rb') as f:
    final = pickle.load(f)
    
#ONLY FOR LIMITED FEATURE MODELS:
top_features = feature_importances.head(27)["feature"].tolist() #27 FEATURES IS BEST

SBFinalistsTrim = SBFinalists[top_features]

# #ensure predictions are the same
# y_pred_test = final.predict(X_test_selected)

# print("Test MSE  : ", mean_squared_error(y_test,y_pred_test))
# print("Test MAE  : ", mean_absolute_error(y_test,y_pred_test))
# print("Test MAPE : ", mean_absolute_percentage_error(y_test,y_pred_test))
# print("Test R2  : ", r2_score(y_test,y_pred_test))

print(final.predict(SBFinalistsTrim))
SBFinalistsTrim.head()

#2025 Predictions are: 18.48pts Eagles, 25.0 PTS Chiefs
# 17 Eagles, 24 Chiefs

[18.487284 25.000254]


,opponents.extra_points.kicks.attempts,record.receiving.targets,record.punt_returns.longest,record.kickoffs.touchbacks,record.defense.sp_assists,record.efficiency.goaltogo.successes,opponents.field_goals.avg_yards,record.defense.passes_defended,opponents.passing.throw_aways,opponents.fumbles.opp_rec_yards,...,record.penalties.penalties,record.efficiency.goaltogo.attempts,record.touchdowns.rush,record.efficiency.redzone.successes,record.rushing.tlost,record.misc_returns.longest_touchdown,record.rushing.longest,opponents.punts.avg_net_yards,opponents.rushing.yards,record.rushing.broken_tackles
0,25,418,31,66,23,25,28.375,90,13,39,...,103,31,29,39,50,NaN,72,41.343,1771,33
1,35,572,55,60,22,25,29.667,71,21,0,...,94,32,15,35,23,0.0,34,41.667,1731,12


## 2025 Predictions are: 18.48pts Eagles, 25.0 PTS Chiefs
# 17 Eagles, 24 Chiefs